In [4]:
# Core imports
import numpy as np
import pandas as pd
import os
#os.listdir()

In [6]:
# Expect the file to be in the same folder as this notebook
df = pd.read_csv('cleaned_retentiondata_case.csv')
print(df.shape)
print(df.describe(include='all'))
#print(df['Outcome'].value_counts())# summary stats
missing_counts = df.isnull().sum()
missing_counts

(5628, 47)
          acct_ref      cust_ref has_dependents    tenure_mo  home_phone  \
count         5628          5628           5628  5628.000000      5628.0   
unique        5613          5628              2          NaN         NaN   
top     ACCT426225  CUST55166391             No          NaN         NaN   
freq             2             1           3957          NaN         NaN   
mean           NaN           NaN            NaN    32.491294         0.0   
std            NaN           NaN            NaN    24.502843         0.0   
min            NaN           NaN            NaN     1.000000         0.0   
25%            NaN           NaN            NaN     9.000000         0.0   
50%            NaN           NaN            NaN    29.000000         0.0   
75%            NaN           NaN            NaN    55.000000         0.0   
max            NaN           NaN            NaN    72.000000         0.0   

        multi_line internet_plan  add_on_security  add_on_backup  \
count   

acct_ref                       0
cust_ref                       0
has_dependents                 0
tenure_mo                      0
home_phone                     0
multi_line                     0
internet_plan                  0
add_on_security                0
add_on_backup                  0
add_on_protection              0
tech_support_std               0
stream_tv                      0
stream_movies                  0
contract_term                  0
e_bill_opt_in                  0
pay_method                     0
monthly_fee                    0
total_billed                   0
left_flag                      0
fiscal_qtr                     0
gender                         0
age_years                      0
is_married                     0
dependents_count               0
referred_friend                0
referrals_count                0
recent_offer                   0
avg_long_dist_fee              0
internet_tech                  0
avg_gb_download                0
premium_su

In [9]:
### For classification problems, first determine the classes of your response variable
#df.columns

print(df["left_flag"].dtype)                     # what type is it? 
print(df["left_flag"].unique())                  # list of unique classes in the response variable

object
['No' 'Yes']


In [11]:
# Map Yes/No to 1/0
df["left_flag"] = df["left_flag"].map({"No": 0, "Yes": 1})

# Define Y and X
Y = df["left_flag"]
X = df.drop(columns=["left_flag"])

In [12]:
from sklearn.model_selection import train_test_split

# Split into train/test sets
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, 
    test_size=0.3,     # 30% test, 70% train
    random_state=99,   # reproducibility
    shuffle=True       # shuffle before splitting
)

# Check shapes
X_train.shape, X_test.shape

((3939, 46), (1689, 46))

In [13]:
# Identify numeric and categorical columns
num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X_train.select_dtypes(exclude=["number"]).columns.tolist()

num_cols, cat_cols

(['tenure_mo',
  'home_phone',
  'multi_line',
  'add_on_security',
  'add_on_backup',
  'add_on_protection',
  'stream_tv',
  'stream_movies',
  'monthly_fee',
  'total_billed',
  'age_years',
  'dependents_count',
  'referrals_count',
  'avg_long_dist_fee',
  'avg_gb_download',
  'premium_support',
  'stream_music',
  'unlimited_data_opt',
  'refunds_total',
  'extra_data_fees_total',
  'long_dist_fees_total',
  'tenure_month_nonzero',
  'avg_monthly_billed',
  'avg_monthly_extra_fees',
  'avg_monthly_long_dist_fee',
  'total_extra_fees',
  'has_internet',
  'count_of_reg_services',
  'count_of_streaming_services',
  'count_of_add_on_services',
  'total_services',
  'service_intensity'],
 ['acct_ref',
  'cust_ref',
  'has_dependents',
  'internet_plan',
  'tech_support_std',
  'contract_term',
  'e_bill_opt_in',
  'pay_method',
  'fiscal_qtr',
  'gender',
  'is_married',
  'referred_friend',
  'recent_offer',
  'internet_tech'])

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

model = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(random_state=99))
])

In [15]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

# Stratified CV preserves class balance in each fold (important for classification)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=10)

# Pipeline with XGBoost model
# NOTE: `preprocess` should already be defined (e.g., a ColumnTransformer)
xgb_pipe = Pipeline([
    ("preprocess", preprocess),
    ("model", XGBClassifier(
        objective="binary:logistic",
        n_jobs=2,           # 1 or 2 threads often runs smoother on student laptops
        eval_metric="auc",  # pairs well with scoring='roc_auc'
        tree_method="hist", # fast CPU histogram algorithm (use 'gpu_hist' if you have a GPU)
        random_state=99
    ))
])

# Hyperparameter grid (reasonable, compact search)
param_grid = {
    "model__n_estimators": [400],
    "model__max_depth":    [4, 8],
    "model__learning_rate":[0.001,0.05],
    "model__subsample":    [1.0],
    "model__colsample_bytree": [1.0],
    "model__min_child_weight": [1]
}

# Cross-validated grid search
xgb_cv = GridSearchCV(
    estimator=xgb_pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="roc_auc",
    n_jobs=-1,
    refit=True,
    verbose=True
)

# Fit 
xgb_cv.fit(X_train, Y_train)

print("Best params selected by CV:", xgb_cv.best_params_)
print("Best CV ROC-AUC:", round(xgb_cv.best_score_, 3))

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params selected by CV: {'model__colsample_bytree': 1.0, 'model__learning_rate': 0.05, 'model__max_depth': 4, 'model__min_child_weight': 1, 'model__n_estimators': 400, 'model__subsample': 1.0}
Best CV ROC-AUC: 0.892


In [16]:
from sklearn.metrics import roc_auc_score, classification_report

test_probs = xgb_cv.predict_proba(X_test)[:, 1]
test_preds = xgb_cv.predict(X_test)

print("Test ROC-AUC:", roc_auc_score(Y_test, test_probs))
print(classification_report(Y_test, test_preds))

Test ROC-AUC: 0.8946655585846222
              precision    recall  f1-score   support

           0       0.85      0.91      0.88      1216
           1       0.72      0.60      0.65       473

    accuracy                           0.82      1689
   macro avg       0.79      0.75      0.77      1689
weighted avg       0.82      0.82      0.82      1689



In [20]:
os.listdir()

['.ipynb_checkpoints',
 'Advertising.csv',
 'advertising_linear_regression.ipynb',
 'bc_data.csv',
 'bc_data_with_risk_segment.csv',
 'cancer_data_dirty.csv',
 'cancer_holdout_no_outcome.csv',
 'cancer_holdout_scored_for_turnin.csv',
 'Churn Group Assignment .ipynb',
 'churn_flow_diagram',
 'cleaned_retentiondata_case.csv',
 'Clustering_Kmeans.ipynb',
 'Credit_Default.csv',
 'customer_data.csv',
 'Data dictionary.xlsx',
 'eda consulting exercise.ipynb',
 'final_columns_table.csv',
 'Group.ipynb',
 'HeavenlyChocolatesInClass.csv',
 'heavenly_chocolate_linear_regression.ipynb',
 'Holdout .ipynb',
 'LASSO_Logistic_regression_inclass.html',
 'Preprocessing & Feature Engineering and Classification Tree Example.ipynb',
 'Random Forest for Classification Problems.ipynb',
 'retentiondata_case.csv',
 'Tree.ipynb',
 'XGBoost Classification Holdout Scoring.ipynb',
 'xgboost_classification.ipynb']

In [24]:
df = pd.read_csv("cleaned_retentiondata_case.csv")
holdout = df.drop(columns=["left_flag"])
holdout.head()

,acct_ref,cust_ref,has_dependents,tenure_mo,home_phone,multi_line,internet_plan,add_on_security,add_on_backup,add_on_protection,...,avg_monthly_billed,avg_monthly_extra_fees,avg_monthly_long_dist_fee,total_extra_fees,has_internet,count_of_reg_services,count_of_streaming_services,count_of_add_on_services,total_services,service_intensity
0,ACCT137932,CUST61880322,No,1,0,0,DSL,0,0,0,...,29.850000,0.0,0.00,0.00,1,1,0,0,1,1.000000
1,ACCT115088,CUST55192304,No,34,0,0,DSL,0,0,0,...,55.573529,0.0,17.09,598.15,1,1,0,0,1,0.029412
2,ACCT623423,CUST16890051,No,2,0,0,DSL,0,0,0,...,54.075000,0.0,10.47,31.41,1,1,0,0,1,0.500000
3,ACCT846960,CUST99388728,No,45,0,0,DSL,0,0,0,...,40.905556,0.0,0.00,0.00,1,1,0,0,1,0.022222
4,ACCT146586,CUST91125265,No,2,0,0,Fiber optic,0,0,0,...,75.825000,0.0,9.12,27.36,1,1,0,0,1,0.500000


In [25]:
holdout.to_csv("retention_holdout_no_left_flag.csv", index=False)

In [27]:
# --- Holdout Scoring Block
holdout = pd.read_csv("retention_holdout_no_left_flag.csv")
X_holdout = holdout[X_train.columns]
best_pipe = xgb_cv.best_estimator_
holdout_probs = best_pipe.predict_proba(X_holdout)[:, 1]
holdout["predicted_churn_prob"] = holdout_probs

holdout.head()

,acct_ref,cust_ref,has_dependents,tenure_mo,home_phone,multi_line,internet_plan,add_on_security,add_on_backup,add_on_protection,...,avg_monthly_extra_fees,avg_monthly_long_dist_fee,total_extra_fees,has_internet,count_of_reg_services,count_of_streaming_services,count_of_add_on_services,total_services,service_intensity,predicted_churn_prob
0,ACCT137932,CUST61880322,No,1,0,0,DSL,0,0,0,...,0.0,0.00,0.00,1,1,0,0,1,1.000000,0.731829
1,ACCT115088,CUST55192304,No,34,0,0,DSL,0,0,0,...,0.0,17.09,598.15,1,1,0,0,1,0.029412,0.027266
2,ACCT623423,CUST16890051,No,2,0,0,DSL,0,0,0,...,0.0,10.47,31.41,1,1,0,0,1,0.500000,0.484148
3,ACCT846960,CUST99388728,No,45,0,0,DSL,0,0,0,...,0.0,0.00,0.00,1,1,0,0,1,0.022222,0.041578
4,ACCT146586,CUST91125265,No,2,0,0,Fiber optic,0,0,0,...,0.0,9.12,27.36,1,1,0,0,1,0.500000,0.927432
